# Libraries

In [51]:
import requests

import numpy as np
import pandas as pd

import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval

import time
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdate

from linearmodels import PooledOLS
import statsmodels.api as sm

pd.set_option('display.max_columns', None) # display all columns in DF


# Import data

In [52]:
df_OLS = pd.read_csv('output_regression.csv', sep = '~')
df_OLS

,date,unique_id,entrance,units_reimbursed_sum,total_amount_reimbursed_sum,first_nda,labeler_name,labeler_name_count,marketing_category,brand_name,generic_name,units_reimbursed,total_amount_reimbursed,price_per_unit_adj
0,1991-01-01,allopurinol-tablet-oral,1.0,5.873886e+05,79096.005000,1.0,"Sun Pharmaceutical Industries, Inc.",2,ANDA,Allopurinol,Allopurinol,130182.716500,17713.875000,0.522740
1,1991-04-01,allopurinol-tablet-oral,0.0,8.547317e+05,120166.020000,1.0,"Sun Pharmaceutical Industries, Inc.",2,ANDA,Allopurinol,Allopurinol,186799.937000,25936.930000,0.523827
2,1991-07-01,allopurinol-tablet-oral,0.0,1.077314e+06,146499.415000,1.0,"Sun Pharmaceutical Industries, Inc.",2,ANDA,Allopurinol,Allopurinol,237011.000000,31794.785000,0.500215
3,1991-10-01,allopurinol-tablet-oral,0.0,9.285903e+05,151942.375000,1.0,"Sun Pharmaceutical Industries, Inc.",2,ANDA,Allopurinol,Allopurinol,146169.074000,30016.535000,0.756787
4,1992-01-01,allopurinol-tablet-oral,0.0,1.196922e+06,163011.650000,1.0,"Sun Pharmaceutical Industries, Inc.",2,ANDA,Allopurinol,Allopurinol,257206.500000,32757.380000,0.459130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15537,2021-01-01,"efavirenz, lamivudine and tenofovir disoproxil...",0.0,8.307000e+03,180802.575000,1.0,Laurus Labs Limited,2,ANDA,"Efavirenz, Lamivudine and Tenofovir Disoproxil...","Efavirenz, Lamivudine and Tenofovir Disoproxil...",927.000000,32373.695000,37.025770
15538,2020-10-01,levothyroxine sodium-capsule-oral,1.0,7.694218e+04,334126.569773,1.0,"Lannett Company, Inc.",2,NDA AUTHORIZED GENERIC,Levothyroxine Sodium,levothyroxine sodium,1555.181818,6465.737273,4.197767
15539,2021-01-01,levothyroxine sodium-capsule-oral,0.0,6.691992e+04,286751.700833,1.0,"Lannett Company, Inc.",2,NDA AUTHORIZED GENERIC,Levothyroxine Sodium,levothyroxine sodium,9775.833333,39310.614167,4.042630
15540,2021-01-01,meloxicam-capsule-oral,1.0,3.416000e+03,75835.250000,1.0,"Lupin Pharmaceuticals, Inc.",2,ANDA,Meloxicam,Meloxicam,3416.000000,75835.250000,22.068262


# Transform Data

In [53]:
df_ind = df_OLS.set_index(["unique_id", "date"])
df_ind
type("date")


str

In [67]:
#df_OLS['date'] = pd.to_datetime(df_OLS['date'], format='%Y-%m-%d')
#df_OLS
type('date')
type('entrance')
type('total_amount_reimbursed')
df_OLS.index = pd.to_datetime(df_OLS.index)
type(df_OLS.index)

#df_OLS['date'] = df_OLS['date'].astype(int)

pandas.core.indexes.datetimes.DatetimeIndex

# Run PooledOLS regression

In [68]:
exog = sm.tools.tools.add_constant(df_ind['entrance'])
endog = df_ind['total_amount_reimbursed']
mod = PooledOLS(endog, exog)


#pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)


# Store values for checking homoskedasticity graphically
#fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
#residuals_pooled_OLS = pooledOLS_res.resids

ValueError: The index on the time dimension must be either numeric or date-like